In [2]:
PROJECT_ID = "formazione-riccardo-zanella" #@param {type: "string"}
REGION = 'us-central1' #@param {type: "string"}
BUCKET_NAME = "bbs-2021-opml4b-explainability" #@param {type:"string"}

In [3]:
# copy data to local
! gsutil cp 'gs://explanations_sample_data/bike-data.csv' ../data/

Copying gs://explanations_sample_data/bike-data.csv...
Skipping attempt to download to filename ending with slash (../data/).
This typically happens when using gsutil to download from a
subdirectory created by the Cloud Console
(https://cloud.google.com/console)

Operation completed over 1 objects/265.3 MiB.                                    


In [4]:
import pandas as pd

In [6]:
data = pd.read_csv('../../data/bike-data.csv')

# Shuffle the data
data = data.sample(frac=1, random_state=2)

# Drop rows with null values
data = data[data['wdsp'] != 999.9]
data = data[data['dewp'] != 9999.9]

# Rename some columns for readability
data = data.rename(columns={'day_of_week': 'weekday'})
data = data.rename(columns={'max': 'max_temp'})
data = data.rename(columns={'dewp': 'dew_point'})

# Drop columns you won't use to train this model
data = data.drop(columns=['start_station_name', 'end_station_name', 'bike_id', 'snow_ice_pellets'])

# Convert trip duration from seconds to minutes so it's easier to understand
data['duration'] = data['duration'].apply(lambda x: float(x / 60))

Let's take a look at the first five rows of your data in the panda dataframe

In [7]:
# Preview the first 5 rows
data.head()

,start_hr,weekday,euclidean,temp,dew_point,wdsp,max_temp,fog,prcp,rain_drizzle,duration
1559391,21,4,582.724880,54.7,46.2,0.0,56.3,0,0.00,0,22.0
1881386,16,2,2358.922742,57.1,48.2,0.0,64.9,0,0.00,0,12.0
703461,23,1,766.066253,69.2,51.6,0.0,80.4,0,0.00,0,12.0
1688076,9,5,2271.877229,52.5,45.4,0.0,63.3,0,0.02,0,13.0
2079007,14,7,2476.718116,50.3,43.5,0.0,56.1,0,0.01,0,28.0


Next, you will separate the data into features ('data') and labels ('labels')

### Split data into train and test sets

You'll split your data into train and test sets using an 80 / 20 train / test split.

In [8]:
# Use 80/20 train/test split
train_size = int(len(data) * .8)
print("Train size: %d" % train_size)
print("Test size: %d" % (len(data) - train_size))

# Split your data into train and test sets
train_data = data[:train_size]

test_data = data[train_size:]

Train size: 757363
Test size: 189341


In [9]:
train_data.to_csv('../../data/train.csv', encoding='utf-8', index=False)
test_data.to_csv('../../data/test.csv', encoding='utf-8', index=False)

In [11]:
# upload
! gsutil -m cp ../../data/train.csv ../../data/test.csv "gs://$BUCKET_NAME/data/tabular_data"

Copying file://../../data/train.csv [Content-Type=text/csv]...
Copying file://../../data/test.csv [Content-Type=text/csv]...

Operation completed over 2 objects/49.4 MiB.                                     
